# Series Collection

Often users of the FRED API will want analyze multiple economic series. This can be done with `FredSeries` alone, but can be tedious and cumbersome. `pyfredapi` offers the `SeriesCollection` class to streamline the process of collecting and munging the data to get ready for plotting and analysis.

A `SeriesCollection` object is a set of `SeriesData` objects. `SeriesCollection` provide helpful methods to:

* List the metadata (frequency, seasonality, units, etc.) of the series in the collection
* Merge series dataframes into a long dataframe
* Merge series dataframes into a wide dataframe by index
* Merge series dataframes into a wide dataframe by date

## Setup

Import and create an instance of `SeriesCollection`

In [1]:
from pyfredapi import SeriesCollection
from rich import print as rprint

cpi_sc = SeriesCollection()

## Collect data

Add data to the collection with `add_series()`. By default the column for the series values will be renamed to the series id.

In [2]:
cpi_series = ["CPIAUCSL", "CPILFESL"]
cpi_sc.add_series(series_ids=cpi_series)

Requesting series CPIAUCSL...
Requesting series CPILFESL...


### Remove series

Remove series to the collection with `drop_series()`.

In [3]:
cpi_sc.drop_series("CPILFESL")

Removed series CPILFESL


## Accessing the data

The `SeriesCollection` is composed of `SeriesInfo` objects. You can access the series either by attribute or by the `data` dictionary.

### Access via attribute

In [4]:
type(cpi_sc.CPIAUCSL)

pyfredapi.api.series.SeriesData

In [5]:
rprint(cpi_sc.CPIAUCSL.info)

SeriesInfo(
    id='CPIAUCSL',
    realtime_start='2022-10-20',
    realtime_end='2022-10-20',
    title='Consumer Price Index for All Urban Consumers: All Items in U.S. City Average',
    observation_start='1947-01-01',
    observation_end='2022-09-01',
    frequency='Monthly',
    frequency_short='M',
    units='Index 1982-1984=100',
    units_short='Index 1982-1984=100',
    seasonal_adjustment='Seasonally Adjusted',
    seasonal_adjustment_short='SA',
    last_updated='2022-10-13 07:37:02-05',
    popularity=96,
    notes='The Consumer Price Index for All Urban Consumers: All Items (CPIAUCSL) is a price index of a basket of 
goods and services paid by urban consumers. Percent changes in the price index measure the inflation rate between 
any two time periods. The most common inflation metric is the percent change from one year ago. It can also 
represent the buying habits of urban consumers. This particular index includes roughly 88 percent of the total 
population, accounting for wage earners, clerical workers, technical workers, self-employed, short-term workers, 
unemployed, retirees, and those not in the labor force.\r\n\r\nThe CPIs are based on prices for food, clothing, 
shelter, and fuels; transportation fares; service fees (e.g., water and sewer service); and sales taxes. Prices are
collected monthly from about 4,000 housing units and approximately 26,000 retail establishments across 87 urban 
areas. To calculate the index, price changes are averaged with weights representing their importance in the 
spending of the particular group. The index measures price changes (as a percent change) from a predetermined 
reference date. In addition to the original unadjusted index distributed, the Bureau of Labor Statistics also 
releases a seasonally adjusted index. The unadjusted series reflects all factors that may influence a change in 
prices. However, it can be very useful to look at the seasonally adjusted CPI, which removes the effects of 
seasonal changes, such as weather, school year, production cycles, and holidays.\r\n\r\nThe CPI can be used to 
recognize periods of inflation and deflation. Significant increases in the CPI within a short time frame might 
indicate a period of inflation, and significant decreases in CPI within a short time frame might indicate a period 
of deflation. However, because the CPI includes volatile food and oil prices, it might not be a reliable measure of
inflationary and deflationary periods. For a more accurate detection, the core CPI (CPILFESL 
(https://fred.stlouisfed.org/series/CPILFESL)) is often used. When using the CPI, please note that it is not 
applicable to all consumers and should not be used to determine relative living costs. Additionally, the CPI is a 
statistical measure vulnerable to sampling error since it is based on a sample of prices and not the complete 
average.\r\n\r\nFor more information on the consumer price indexes, see:  \r\nBureau of Economic Analysis. "CPI 
Detailed Report." (https://www.bls.gov/cpi/) 2013.  \r\nHandbook of Methods 
(https://www.bls.gov/opub/hom/pdf/cpihom.pdf)  \r\nUnderstanding the CPI: Frequently Asked Questions 
(https://www.bls.gov/cpi/questions-and-answers.htm)'
)

In [6]:
cpi_sc.CPIAUCSL.data.head()

,date,CPIAUCSL
0,1947-01-01,21.48
1,1947-02-01,21.62
2,1947-03-01,22.00
3,1947-04-01,22.00
4,1947-05-01,21.95


### Access via dictionary

In [7]:
cpi_sc.data["CPIAUCSL"].data.head()

,date,CPIAUCSL
0,1947-01-01,21.48
1,1947-02-01,21.62
2,1947-03-01,22.00
3,1947-04-01,22.00
4,1947-05-01,21.95


## Rename series in the collection

### Rename on add

You can rename the series when adding them to the collection. Renaming can be done with a dictionary mapping the series id to the new name, or with a function which parses the series title into the new name.

In [8]:
# Rename with a dictionary
new_names = {
    "CPIAUCSL": "cpi_all_items",
    "CPILFESL": "cpi_all_items_less_food_and_energy",
}

cpi_sc = SeriesCollection()
cpi_sc.add_series(series_ids=cpi_series, rename=new_names)

Requesting series CPIAUCSL...
Requesting series CPILFESL...


In [9]:
cpi_sc.CPIAUCSL.data.head()

,date,cpi_all_items
0,1947-01-01,21.48
1,1947-02-01,21.62
2,1947-03-01,22.00
3,1947-04-01,22.00
4,1947-05-01,21.95


In [10]:
cpi_sc.CPILFESL.data.head()

,date,cpi_all_items_less_food_and_energy
0,1957-01-01,28.5
1,1957-02-01,28.6
2,1957-03-01,28.7
3,1957-04-01,28.8
4,1957-05-01,28.8


### Rename after add

You can rename series in the collection with the `rename_series` method. Works the same way as renaming on add.

In [11]:
def parse_cpi_title(title: str) -> str:
    """Parse CPI series title into a readable label."""
    return (
        title.lower()
        .replace("consumer price index", "cpi ")
        .replace(" for all urban consumers: ", "")
        .replace(" in u.s. city average", "")
        .replace(" ", "_")
    )


cpi_sc.rename_series(rename=parse_cpi_title)

In [12]:
cpi_sc.CPIAUCSL.data.head()

,date,cpi_all_items
0,1947-01-01,21.48
1,1947-02-01,21.62
2,1947-03-01,22.00
3,1947-04-01,22.00
4,1947-05-01,21.95


## List metadata

`SeriesCollection` has a number of list methods to print out the metadata of the series in the collection.

### Series in the collection

In [13]:
cpi_sc.list_series()

CPIAUCSL: Consumer Price Index for All Urban Consumers: All Items in U.S. City Average

CPILFESL: Consumer Price Index for All Urban Consumers: All Items Less Food and Energy in U.S. City Average

### Frequency

In [14]:
cpi_sc.list_frequency()

All series are Monthly


### Seasonality

In [15]:
cpi_sc.list_seasonality()

All series are Seasonally Adjusted


### Units

In [16]:
cpi_sc.list_units()

All series are that are measured in Index 1982-1984=100


### Dates

In [17]:
cpi_sc.list_end_date()

All series end on 2022-09-01


In [18]:
cpi_sc.list_start_date()

───────────────────────────────────────── Series that start on 1957-01-01 ─────────────────────────────────────────

CPILFESL: Consumer Price Index for All Urban Consumers: All Items Less Food and Energy in U.S. City Average

───────────────────────────────────────── Series that start on 1947-01-01 ─────────────────────────────────────────

CPIAUCSL: Consumer Price Index for All Urban Consumers: All Items in U.S. City Average

## Merge data

`SeriesCollection` supports merging the data into long and wide formats. By default the series ID will be used as the column name or observation label.

### Merge long

Merge the series in the collection into a long pandas dataframe.

In [19]:
cpi_long = cpi_sc.merge_long()
cpi_long

,date,value,series
0,1947-01-01,21.480,cpi_all_items
1,1947-02-01,21.620,cpi_all_items
2,1947-03-01,22.000,cpi_all_items
3,1947-04-01,22.000,cpi_all_items
4,1947-05-01,21.950,cpi_all_items
...,...,...,...
1693,2022-05-01,292.289,cpi_all_items_less_food_and_energy
1694,2022-06-01,294.354,cpi_all_items_less_food_and_energy
1695,2022-07-01,295.275,cpi_all_items_less_food_and_energy
1696,2022-08-01,296.950,cpi_all_items_less_food_and_energy


### Merge as-of

Merge the series in the collection into a wide pandas dataframe based on nearest date. Must define a base series. The base series defines the set of dates to serve of the basis of joining.

In [20]:
cpi_asof = cpi_sc.merge_asof(base_series_id="CPIAUCSL")
cpi_asof.tail()

,date,cpi_all_items,cpi_all_items_less_food_and_energy
904,2022-05-01,291.474,292.289
905,2022-06-01,295.328,294.354
906,2022-07-01,295.271,295.275
907,2022-08-01,295.620,296.950
908,2022-09-01,296.761,298.660


### Merge wide

Merge the series in the collection into a wide pandas dataframe. Only works if all the series in the collection share the same date index.

In [21]:
cpi_wide = cpi_sc.merge_wide()
cpi_wide.tail()

,date,cpi_all_items,cpi_all_items_less_food_and_energy
904,2022-05-01,291.474,292.289
905,2022-06-01,295.328,294.354
906,2022-07-01,295.271,295.275
907,2022-08-01,295.620,296.950
908,2022-09-01,296.761,298.660
